In [28]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [29]:
xx = QQ['xx'].0

In [30]:
Q.<alpha> = NumberField(xx^2-5)

In [31]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
P

Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in alpha with defining polynomial xx^2 - 5

In [32]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [33]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [34]:
line = Line([y, z])

In [35]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [36]:
eqn_SE3 = P(2*eqn_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}) )
sing_cubics_SE3 = P(sing_cubics_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}))

In [37]:
eqn_SE4 = -P((c+e) * eqn_SE3.subs({f:c*(3*c-e)/(c+e)}) / (4*c))
sing_cubics_SE4 = P((c+e)^7*sing_cubics_SE3.subs({f:c*(3*c-e)/(c+e)}))

In [38]:
eqn_SE10 = P(eqn_SE4.subs({e : (2-alpha)*c})/(c^2*(alpha-3)))
sing_cubics_SE10 = P(sing_cubics_SE4.subs({e : (2-alpha)*c}))

In [39]:
SE10 = Cubic(eqn_SE10, line, sing_cubics_SE10.factor())

In [40]:
def apply_proj_to_eck(proj, eck):
    new_indices = []
    for i in range(len(eck)):
        new_indices.append(eck.index(eck[i]*proj)+1)
    return new_indices

In [41]:
    def find_admissible_permutations(self):
        with open('all_permutations.pickle', 'rb') as ff:
            all_permutations = pickle.load(ff) 
        adm_perm = []
        keys = list(self.cl_lines.keys())
        labels = [sorted(el.labels) for el in self.tritangent_planes if el.conditions == 0]
        for perm in all_permutations:
            flag = True
            for triple in labels:
                new_triple = []
                for label in triple:
                    new_triple.append(perm[keys.index(label)])
                if sorted(new_triple) not in labels:
                    flag = False
                    break
            if flag is True:
                adm_perm.append(perm)
        return adm_perm

In [42]:
labels = [sorted(pl.labels) for pl in SE10.tritangent_planes if pl.conditions == 0]
keys = list(SE10.cl_lines.keys())
labels

[['E1', 'F12', 'G2'],
 ['E1', 'F14', 'G4'],
 ['E2', 'F23', 'G3'],
 ['E2', 'F24', 'G4'],
 ['E3', 'F13', 'G1'],
 ['E3', 'F23', 'G2'],
 ['E4', 'F14', 'G1'],
 ['E4', 'F34', 'G3'],
 ['F12', 'F34', 'F56'],
 ['F13', 'F24', 'F56']]

In [43]:
adm_perm = SE10.find_admissible_permutations()

In [44]:
len(adm_perm)

240

In [18]:
adm_perm.index(['E1', 'E2', 'E3', 'E4', 'E6', 'E5', 'G1', 'G2', 'G3', 'G4', 'G6', 'G5', 'F12', 'F13', 'F14', 'F16', 'F15', 'F23', 'F24', 'F26', 'F25', 'F34', 'F36', 'F35', 'F46', 'F45', 'F56'])

0

In [45]:
resulting_L_sets = []
conjugate_perm = []
for perm in adm_perm:
    perm_L_set = [perm[list(SE10.cl_lines.keys()).index(label)] for label in ['E1', 'G4', 'E2', 'G3', 'E3']]
    if perm_L_set not in resulting_L_sets:
        resulting_L_sets.append(perm_L_set)
    else:
        conjugate_perm.append(perm)

In [46]:
len(conjugate_perm)

120

In [48]:
adm_proj = SE10.find_admissible_projectivities()

In [52]:
simm_SE10 = SE10.find_simmetries(adm_proj)

In [53]:
len(adm_proj)

120

In [54]:
perms = []
normal_perm = [perm for perm in adm_perm if perm not in conjugate_perm]
print(len(normal_perm))
for simm in simm_SE10:
    perm = normal_perm[adm_proj.index(simm)]
    perms.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())
    #perm = conjugate_perm[adm_proj.index(simm)]
    #perms.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())

120


In [55]:
len(perms)

120

In [56]:
Gr = PermutationGroup(perms)

In [59]:
Gr.structure_description()

'C2 x S5'